In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import os

# Initialize the WebDriver
driver = webdriver.Chrome()

# Define the base URL for the main page
base_url = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&company=masterworks&owner=include&match=&start={}&count=40&hidefilings=0"

# Function to scrape links to individual company pages from the first page
def get_company_links(driver):
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//table[@class="tableFile2"]/tbody/tr'))
    )
    rows = driver.find_elements(By.XPATH, '//table[@class="tableFile2"]/tbody/tr')
    company_links = []
    for row in rows[1:]:  # skip the header row
        cols = row.find_elements(By.TAG_NAME, 'td')
        if len(cols) >= 2:
            cik_element = cols[0].find_element(By.TAG_NAME, 'a')
            company_cik = cik_element.text
            company_url = cik_element.get_attribute('href')
            company_name = cols[1].text.split("\n")[0]
            if "masterworks" in str.lower(company_name):  # filter based on the presence of "Masterworks" in the company name
                company_links.append((company_cik, company_name))
    return company_links

# Initialize a list to hold all the scraped data
all_data = []

# Start scraping from the main page and handle pagination through URL
start = 0
while True:
    driver.get(base_url.format(start))
    try:
        company_links = get_company_links(driver)
    except Exception as e:
        print(f"No more companies: {e}")
        break
    
    if not company_links:
        break
    
    all_data.extend(company_links)

    start += 40

# Close the WebDriver
driver.quit()

# Convert the scraped data into a DataFrame
df = pd.DataFrame(all_data, columns=['CIK', 'Company Name'])

# Add a space after each comma in the 'Company Name' column
df['Company Name'] = df['Company Name'].apply(lambda x: ', '.join([name.strip() for name in x.split(',')]))

# Ensure CIK column is saved as text to preserve leading zeros
df['CIK'] = df['CIK'].astype(str)

# Save the data to a CSV file
os.makedirs('../data', exist_ok=True)
df.to_csv('../data/masterworks_entity_list.csv', index=False)

print(f"Data scraping complete. All data has been saved to 'masterworks_entity_list.csv'.")


No more companies: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=127.0.6533.100)
Stacktrace:
0   chromedriver                        0x000000010530d0b8 cxxbridge1$str$ptr + 1887276
1   chromedriver                        0x0000000105305794 cxxbridge1$str$ptr + 1856264
2   chromedriver                        0x0000000104f1482c cxxbridge1$string$len + 88524
3   chromedriver                        0x0000000104eefe50 core::str::slice_error_fail::he7b2aa4898bc357e + 3908
4   chromedriver                        0x0000000104f7d574 cxxbridge1$string$len + 517908
5   chromedriver                        0x0000000104f8fdd8 cxxbridge1$string$len + 593784
6   chromedriver                        0x0000000104f4d474 cxxbridge1$string$len + 321044
7   chromedriver                        0x0000000104f4e0e4 cxxbridge1$string$len + 324228
8   chromedriver                        0x00000001052d4a9c cxxbridge1$str$ptr + 1656336
9   chrome